In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Normalizer
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler



In [2]:

# Create a Spark session
spark = SparkSession.builder.appName("VectorAssemblerExample").getOrCreate()

23/12/18 18:29:02 WARN Utils: Your hostname, alvaro-ThinkPad-X1-Carbon-Gen-10 resolves to a loopback address: 127.0.1.1; using 10.156.4.72 instead (on interface wlp0s20f3)
23/12/18 18:29:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/18 18:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/18 18:29:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/18 18:29:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:


# Sample data
data = [(1.60193653, -1.8679101),
        (-1.1328963, -1.9607465),
        (2.40675869, -1.5994823),
        (0.09330145, -2.9446696),
        (1.3795901, -2.5489864),
        (-0.42065496, -2.8165693),
        (0.55753398, -2.0145494),
        (1.3066549, -2.3208153),
        (0.66224722, -0.9406476),
        (1.19072851, -2.4178092),
        (4.67961769, 1.6375689),
        (5.03015133, 1.4575724),
        (6.1003413, 2.1673923),
        (4.20259176, 1.8237144),
        (4.93339445, 1.8983999),
        (6.70975052, 1.5899655),
        (5.01461979, 1.6051478),
        (5.00005277, 1.6855351),
        (4.12926186, 2.0582398)]

columns = ["x1", "x2"]

df2 = spark.createDataFrame(data, columns)
# Split the data into training and testing sets
split = df2.randomSplit([0.7, 0.3])  # ESTO no tiene sentido en clustering
training = split[0]
test = split[1]

In [6]:

# Create the Vector Assembler
assembler = VectorAssembler(inputCols=["x1", "x2"], outputCol="features")

# Create a Normalizer
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

# Create a K-Means model
kmeans = KMeans(featuresCol="features", k=2)

# Create a Pipeline
pipeline = Pipeline(stages=[assembler,normalizer, kmeans])

# Fit the Pipeline on the training data
model = pipeline.fit(df2)

# Transform the test data and show the results
transformed_data = model.transform(test)
transformed_data.show(truncate=False)

+----------+----------+-----------------------+----------------------------------------+----------+
|x1        |x2        |features               |normFeatures                            |prediction|
+----------+----------+-----------------------+----------------------------------------+----------+
|1.19072851|-2.4178092|[1.19072851,-2.4178092]|[0.3299753544767584,-0.6700246455232416]|0         |
|5.01461979|1.6051478 |[5.01461979,1.6051478] |[0.7575220310717887,0.24247796892821125]|1         |
+----------+----------+-----------------------+----------------------------------------+----------+

